# Issues with concurrency

With multiple tasks executing simultaneously, it's possible for unplanned events to arise. 

Because these are unplanned events, they can be difficult to diagnose, much less correct, so it's best to avoid them altogether (if possible). 

These unplanned events arise whenever mutliple tasks are using a shared resource. It is useful to share resources in theory, because then some tasks can be completed more quickly (or differently), but we have to be careful!

The code below shows code which may behave unpredictably.

In [9]:
from threading import Thread

counter = 0
n = 10_000 # try bigger/smaller values

def inc():
   global counter
   for i in range(100000):
       counter += 1
        # read in the value of counter
            # read in the value of counter
            # change the value of counter
        # change the value of counter
            # read out the value of counter
        # read out the value counter

def dec():
   global counter
   for i in range(100000):
       counter -= 1

t1 = Thread(target=inc)
t2 = Thread(target=dec)
t1.start()
t2.start()
t1.join()
t2.join()

print( "Counter =", counter)


Counter = 0


This happens because there are multiple ways for t1 and t2 to run. They can run in any order, but it is also possible for t1 to be in the middle of running while t2 begins, or vice versa. This can happen with processes as well.

In [11]:
%%file my_counter.py
def load():
    try:
        with open("counter.txt") as f:
            return int(f.read())
    except:
        return 0

def save(num):
    with open("counter.txt", "w") as f:
        f.write(str(num))
        
def inc():
    for i in range(1000):
        n = load()
        n = n + 1
        save(n)

def dec():
    for i in range(1000):
        n = load()
        n = n - 1
        save(n)

Overwriting my_counter.py


In [12]:
# The code in this cell just insures that we don't have to restart the kernel if we modify my_counter.py
import importlib

try:
    importlib.reload(mc)
except:
    pass

In [15]:
import my_counter as mc

mc.save(0)
print(mc.load())
mc.inc()
print(mc.load())
mc.dec()
print(mc.load())

0
1000
0


In [19]:
from multiprocessing import Process
import my_counter as mc

mc.save(0)

p1 = Process(target=mc.inc)
p2 = Process(target=mc.dec)

p1.start()
p2.start()

p1.join()
p2.join()

print("Counter =", mc.load())



Counter = -39


# Locks

One of the mechanisms we use is a **lock**. We can identify all the parts of the program where a shared resource could be accessed, and ensure that these cannot run concurrently. 

We do this by requiring that each of these parts acquire a lock before running. Any other threads or processes which would like to run, but which doesn't have a lock is blocked from running until the one holding the lock gets through their code.

In [20]:
from threading import Lock

lock = Lock()
counter = 0
n = 1000


# manual acquisition and release of locks
def inc():
    global counter
    for i in range(n):
        lock.acquire()
        counter += 1
        lock.release() 

#automatic release of locks
def dec():
    global counter
    for i in range(n):
        with lock: 
            counter -= 1
            
counter = 0

t1 = Thread(target=inc)
t2 = Thread(target=dec)

t1.start()
t2.start()

t1.join()
t2.join()

print ("Counter =", counter)

Counter = 0


Processes and locks

In [21]:
%%file my_counter.py

def load():
    try:
        with open("counter.txt") as f:
            return int(f.read())
    except:
        return 0
    
def save(num):
    with open("counter.txt", "w") as f:
        f.write(str(num))
        
def inc(lock):
    for i in range(1,1000):
        with lock:
            n = load()
            n = n + 1
            save(n)

def dec(lock):
    for i in range(1,1000):
        with lock:
            n = load()
            n = n - 1
            save(n)

Overwriting my_counter.py


In [22]:
from multiprocessing import Process, Lock
import my_counter as mc

lock = Lock()
mc.save(0)
p1 = Process(target = mc.inc, args=(lock,))
p2 = Process(target = mc.dec, args=(lock,))

p1.start()
p2.start()
p1.join()
p2.join()

print("counter=", mc.load())

counter= 0


The next thing we consider is what happens when we have separate locks for separate shared resources. Is it possible for these to interact in a way that's undesirable?

In [ ]:
`